In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
trade_type = "exports"

my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"
# This is my key. I'm nice and I have it posted. If you will be doing more with this
# please get your own key!

In [3]:
def census_trade(url, trade_type, country, product_level):
    
    r = requests.get(url) 
    
    print(r)
    
    df = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

    df.columns = r.json()[0]

    df.time = pd.to_datetime(df.time, format="%Y-%m")
    # This is so I can call this correctly...
    
    if trade_type == "imports":
        
        trade_label = country + "_" + trade_type
        
        df[trade_label] = df["CON_VAL_MO"].astype(float)
        
        df[product_level] = df["I_COMMODITY"].astype(str)
        
        df.drop(["CON_VAL_MO", "I_COMMODITY", "COMM_LVL"], axis = 1, inplace = True)
        
    if trade_type == "exports":
    
        trade_label = country + "_" + trade_type
        
        df[trade_label] = df["ALL_VAL_MO"].astype(float)

        df[product_level] = df["E_COMMODITY"].astype(str)
        
        df.drop(["ALL_VAL_MO", "E_COMMODITY", "COMM_LVL"], axis = 1, inplace = True)
    
    return df

In [4]:
end_use = "hs?get=E_COMMODITY,ALL_VAL_MO"

url = "https://api.census.gov/data/timeseries/intltrade/exports/" + end_use 
url = url + my_key + "&time==from+2017-01" + "&COMM_LVL=HS4"

url = url + "&CTY_CODE=5700"

df = census_trade(url, trade_type, "china", "hs4")

###########################################################################

#end_use = "hs?get=E_COMMODITY,ALL_VAL_MO"

#url = "https://api.census.gov/data/timeseries/intltrade/exports/" + end_use 
#url = url + my_key + "&time==from+2020-01" + "&COMM_LVL=HS4"

#url = url + "&CTY_CODE=5700"

#df = df.append(census_trade(url, trade_type, "china", "hs4"))

<Response [200]>


In [5]:
df.tail()

,time,CTY_CODE,china_exports,hs4
42504,2020-03-01,5700,515668.0,9028
42505,2020-03-01,5700,1134151.0,9029
42506,2020-03-01,5700,79625044.0,9030
42507,2020-03-01,5700,70801509.0,9031
42508,2020-03-01,5700,18000630.0,9032


In [6]:
dfproducts = pd.read_csv(".\\data"+ "\\annex-6-1.csv", dtype = {"hs4": str})

In [7]:
df_phaseone = df.merge(dfproducts, left_on = "hs4", right_on = "hs4", how = "left", indicator = True)

In [8]:
df_phaseone.high_catagory = df_phaseone.high_catagory.fillna("not in aggreement")

In [9]:
out_file = ".\\data"+ "\\phaseone-tradedata.parquet"

pq.write_table(pa.Table.from_pandas(df_phaseone), out_file)

In [10]:
df_phaseone.set_index("time", inplace = True)

In [11]:
grp = df_phaseone.loc["2017"].groupby("high_catagory")

benchmarks = grp.agg({"china_exports": "sum"})

benchmarks.columns = ["2017 Values"]

In [12]:
grp = df_phaseone.loc["2020"].groupby("high_catagory")

current = grp.agg({"china_exports": "sum"})

current.columns = ["2020 Values"]

In [13]:
benchmarks = benchmarks.merge(current, left_index = True, right_index = True)

In [14]:
benchmarks["Goals"] = 32900000000

benchmarks.iloc[1,2] = 12500000000

benchmarks.iloc[2,2] = 18500000000

benchmarks.iloc[3,2] = np.nan

In [16]:
out_file = ".\\data"+ "\\phaseone-goals.parquet"

pq.write_table(pa.Table.from_pandas(benchmarks), out_file)

In [17]:
benchmarks

,2017 Values,2020 Values,Goals
high_catagory,,,
1. Manufactured Goods,5.023535e+10,1.088696e+10,3.290000e+10
2. Agriculture,2.125611e+10,3.177704e+09,1.250000e+10
3. Energy,1.592617e+10,8.891867e+08,1.850000e+10
not in aggreement,5.026658e+10,7.558639e+09,NaN
